# Explore signal/price movement relationship

In [1]:
import datetime
import importlib.util
import sys
from tradingstrategy.client import Client
from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.execution_context import notebook_execution_context


from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.strategy_module import read_strategy_module

strategies_folder = get_default_strategies_path()
strategy_path = strategies_folder / "spot-and-short-momentum-tuned.py"

strategy_mod = read_strategy_module(strategy_path)

print(f"Backtesting for strategy {strategy_path}, engine version {strategy_mod.trading_strategy_engine_version}")

client = Client.create_jupyter_client()

# contains start_at, end_at backtesting range
universe_options = strategy_mod.get_universe_options()

strategy_universe = strategy_mod.create_trading_universe(
    datetime.datetime.utcnow(),
    client,
    notebook_execution_context,
    universe_options,
)

data_universe = strategy_universe.data_universe

Backtesting for strategy /Users/moo/code/ts/trade-executor/strategies/spot-and-short-momentum-tuned.py, engine version 0.3
Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


## Github mode

Set chart output mode to static images that Github website notebook viewer can display.

In [ ]:


from tradeexecutor.utils.notebook import OutputMode, setup_charting_and_output


setup_charting_and_output(OutputMode.static, image_format="png")

## Explore signal vs price change

Create a function `calculate_signal_vs_profit` which calculates 
- Signal (naive momentum)
- Profit: the last and the best future price we can get
- Allows us to play around with different time windows
- Split between shorts and longs


In [2]:
import pandas as pd
import numpy as np
from pandas.tseries.frequencies import to_offset

from tradingstrategy.utils.forward_fill import forward_fill
from tradingstrategy.chain import ChainId
from tradingstrategy.pair import DEXPair

def calculate_signal_vs_profit(df, signal_window: pd.Timedelta, profit_window: pd.Timedelta) -> pd.DataFrame:
    """Calculate signals and profits for all incoming candles."""
    
    # Create entries for past price to be used for signal
    # and future price (used for the price correlation)
    signal_offset = to_offset(signal_window)
    profit_offset = to_offset(profit_window)

    df["prev"] = df["close"].shift(freq=signal_offset)
    df["next"] = df["open"].shift(freq=-profit_offset)

    # Calculate signal from the past and price difference to the future
    df["signal"] = (df["prev"] - df["open"]) / df["open"]
    df["price_diff"] = (df["next"] - df["open"]) / df["open"]  # Get the profit on the final day of profit window

    # On negative signals, we go short.
    # On zero signal and lack of data set side to NA
    df["side"] = pd.NA
    df.loc[df["signal"] > 0, "side"] = "long"
    df.loc[df["signal"] < 0, "side"] = "short"

    # Max and min price wihtin the profit window will determine the profit for longs and shorts respective
    df["max_future_price"] = df["close"].rolling(profit_window.days).max().shift(-profit_window.days) # Get the max profit on the profit window, assuming take profit %
    df["min_future_price"] = df["close"].rolling(profit_window.days).min().shift(-profit_window.days) # Get the max profit on the profit window, assuming take profit %    
    
    # Calculate profit separately for longs and shorts
    # using Pandas Mask
    # https://stackoverflow.com/a/33770421/315168
    #
    # We calculate both profit after X time,
    # and also max take profit, assuming
    # we could do a perfect trailing stop loss
    #
    longs = (df["side"] == "long")
    shorts = (df["side"] == "short")
    df.loc[longs, "profit"] = df["price_diff"]
    df.loc[shorts, "profit"] = -df["price_diff"]
    df.loc[longs, "profit_max"] = (df["max_future_price"] - df["open"]) / df["open"]  # Get the profit based on max price
    df.loc[shorts, "profit_max"] = -(df["min_future_price"] - df["open"]) / df["open"]  # Get the profit based on max price

    df.loc[longs, "desc"] = df.agg('{0[pair]} long'.format, axis=1)
    df.loc[shorts, "desc"] = df.agg('{0[pair]} short'.format, axis=1)

    
    return df
     


Create a function `calculate_signal_vs_price_for_pair` which calculates 
- Calculates the signal vs. for certain trading pair
- Bundle few different pairs to the same `DataFrame` so we can examine them together


In [3]:
def calculate_signal_vs_price_for_pair(pair: DEXPair) -> pd.DataFrame:
    """Calculate signal vs. profit ratio for an individual pair."""
    df = data_universe.candles.get_candles_by_pair(pair)
    assert df is not None

    df = df.copy()

    # Make sure there are no gaps in the data
    df = forward_fill(
        df, 
        freq=data_universe.time_bucket.to_frequency(), 
        columns=("open", "high", "low", "close")
    )

    df["pair"] = pair.get_ticker()
    df = calculate_signal_vs_profit(
        df,
        #signal_window=pd.DateOffset(days=7),
        #profit_window=pd.DateOffset(days=7),
        signal_window=pd.Timedelta(days=8),
        profit_window=pd.Timedelta(days=8),
    )

    
    return df

pairs = data_universe.pairs
eth = calculate_signal_vs_price_for_pair(pairs.get_pair_by_human_description((ChainId.polygon, "uniswap-v3", "WETH", "USDC")))
link = calculate_signal_vs_price_for_pair(pairs.get_pair_by_human_description((ChainId.polygon, "uniswap-v3", "LINK", "USDC")))
matic = calculate_signal_vs_price_for_pair(pairs.get_pair_by_human_description((ChainId.polygon, "uniswap-v3", "WMATIC", "USDC")))

df = pd.concat([eth, link, matic])

pd.set_option('display.min_rows', 36)
display(df)

,open,high,low,close,pair,prev,next,signal,price_diff,side,max_future_price,min_future_price,profit,profit_max,desc
timestamp,,,,,,,,,,,,,,,
2022-07-07,1184.941609,1252.561586,1178.206991,1237.621415,WETH-USDC,NaN,1192.310699,NaN,0.006219,<NA>,1230.956604,1036.863378,NaN,NaN,NaN
2022-07-08,1237.497665,1271.872162,1194.458684,1215.787741,WETH-USDC,NaN,1230.956604,NaN,-0.005286,<NA>,1356.200105,1036.863378,NaN,NaN,NaN
2022-07-09,1215.544620,1230.956604,1204.654343,1216.882388,WETH-USDC,NaN,1356.200105,NaN,0.115714,<NA>,1356.200105,1036.863378,NaN,NaN,NaN
2022-07-10,1216.882388,1218.099818,1153.839213,1166.834199,WETH-USDC,NaN,1337.747773,NaN,0.099324,<NA>,1581.982883,1036.863378,NaN,NaN,NaN
2022-07-11,1166.950882,1169.287002,1091.985483,1096.033110,WETH-USDC,NaN,1581.982883,NaN,0.355655,<NA>,1581.982883,1036.863378,NaN,NaN,NaN
2022-07-12,1096.142714,1096.910244,1034.067762,1036.863378,WETH-USDC,NaN,1543.697132,NaN,0.408299,<NA>,1581.982883,1114.601072,NaN,NaN,NaN
2022-07-13,1036.863378,1116.497411,1011.466657,1114.601072,WETH-USDC,NaN,1520.867688,NaN,0.466797,<NA>,1581.982883,1192.429930,NaN,NaN,NaN
2022-07-14,1114.712532,1208.515218,1073.579544,1192.429930,WETH-USDC,NaN,1575.352819,NaN,0.413237,<NA>,1581.982883,1230.956604,NaN,NaN,NaN
2022-07-15,1192.310699,1284.525595,1181.392288,1230.956604,WETH-USDC,1237.621415,1534.770049,0.038002,0.287223,long,1581.982883,1337.480264,0.287223,0.326821,WETH-USDC long


## Plot signal vs. price samples

Different scatter charts to examine if there is a correlation between the signal and the profit.

In [4]:
import plotly.express as px
fig = px.scatter(df, x="signal", y="profit_max", trendline="ols", color="desc", title="Max take profit")
fig.show()

fig = px.scatter(df, x="signal", y="profit", trendline="ols", color="desc", title="Profit last day")
fig.show()

### Long only vs. short 

Examine trade types separately to better see if one side is clearly superior.

In [5]:
df_long_only = df.loc[df["side"] == "long"]

fig = px.scatter(df_long_only, x="signal", y="profit_max", trendline="ols", color="desc", title="Long only max take profit")
fig.show()

fig = px.scatter(df_long_only, x="signal", y="profit", trendline="ols", color="desc", title="Long only profit last day")
fig.show()

In [6]:
df_short_only = df.loc[df["side"] == "short"]

fig = px.scatter(df_short_only, x="signal", y="profit_max", trendline="ols", color="desc", title="Short only max take profit")
fig.show()

fig = px.scatter(df_short_only, x="signal", y="profit", trendline="ols", color="desc", title="Short only profit last day")
fig.show()

### Filtered signal

Remove low signals below a certain threshold and see if it affects the correlation between the signal and the profit.

In [7]:

threshold = 0.05

filtered_df = df.loc[abs(df["signal"]) >= threshold]

fig = px.scatter(filtered_df, x="signal", y="profit_max", trendline="ols", color="desc", title=f"Filtered signal at {threshold} max take profit")
fig.show()

fig = px.scatter(filtered_df, x="signal", y="profit", trendline="ols", color="desc", title=f"Filtered signal at {threshold} profit last day")
fig.show()